In [6]:
import pandas as pd
import numpy as np
import sqlite3 as lite
import scipy
from scipy.stats import spearmanr
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [8]:
conn = lite.connect("nigeria.db")

In [42]:
#zone, state, lga, sector, ea, hhid, plotid, cropid, cropname, cropcode, sa3iq3, sa3iq5a, sa3iq5b, sa3iq5c, sa3iq6i, sa3iq6ii, sa3iq6a, sa3iq6d1 
#sa3iiq3, sa3iiq4, sa3iiq5a, sa3iiq5b, sa3iiq6, sa3iiq11a, sa3iiq11b, sa3iiq12a, sa3iiq12b, sa3iiq13a, sa3iiq13b, sa3iiq14a, sa3iiq14b, sa3iiq15a, sa3iiq15b, sa3iiq16a, sa3iiq16b, sa3iiq17a, sa3iiq17b, sa3iiq18a, sa3iiq18b, sa3iiq19, sa3iiq20a, sa3iiq20b, sa3iiq21 
harv_data = pd.read_sql_query("select * from secta3i LEFT JOIN secta3ii ON secta3i.hhid = secta3ii.hhid", conn)
harv_data = pd.DataFrame(harv_data)
harv_data.head()

,index,zone,state,lga,sector,ea,hhid,plotid,cropid,cropname,...,sa3iiq21,sa3iiq22a,sa3iiq22b,sa3iiq23a,sa3iiq23b,sa3iiq24,sa3iiq25,sa3iiq25_os,sa3iiq26,sa3iiq27
0,0,4,1,105,2,0,10009,1,1,CASSAVA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,0,4,1,105,2,0,10009,1,1,CASSAVA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,0,4,1,105,2,0,10009,1,1,CASSAVA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
3,1,4,1,105,2,0,10009,1,2,MAIZE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4,1,4,1,105,2,0,10009,1,2,MAIZE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [31]:
##o for rows, 1 for columns
na_vals = harv_data.apply(pd.isnull, axis = 1)
na_vals.describe()

,index,zone,state,lga,sector,ea,hhid,plotid,cropid,cropname,...,sa3iiq21,sa3iiq22a,sa3iiq22b,sa3iiq23a,sa3iiq23b,sa3iiq24,sa3iiq25,sa3iiq25_os,sa3iiq26,sa3iiq27
count,43384,43384,43384,43384,43384,43384,43384,43384,43384,43384,...,43384,43384,43384,43384,43384,43384,43384,43384,43384,43384
unique,1,1,1,1,1,1,1,1,1,2,...,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
freq,43384,43384,43384,43384,43384,43384,43384,43384,43384,42406,...,40633,40629,42841,40678,42743,40629,40629,42972,40632,43300


In [34]:
pd.read_sql_query("select * from secta3i group by hhid", conn)

,index,zone,state,lga,sector,ea,hhid,plotid,cropid,cropname,...,sa3iq6ii_os,sa3iq6a,sa3iq6b,sa3iq6c1,sa3iq6c2,sa3iq6d1,sa3iq6d2,sa3iq6d2_os,sa3iq6e1,sa3iq6e2
0,8,4,1,105,2,0,10009,3,4,BANANA,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,12,4,1,104,2,1364,10011,1,4,PLANTAIN,...,None,5000.0,2.0,NaN,NaN,16.0,111.0,None,1.0,NaN
2,20,4,1,104,2,1364,10012,2,3,THREE LEAVE YAM,...,None,4500.0,2.0,NaN,NaN,13.0,80.0,None,2.0,4.0
3,23,4,1,104,2,1364,10013,2,1,CASSAVA,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4,27,4,1,104,2,0,10015,2,2,MAIZE,...,None,2400.0,1.0,9.0,2015.0,NaN,NaN,None,2.0,NaN
5,32,4,1,104,2,1364,10016,1,5,MAIZE,...,None,1800.0,1.0,10.0,2015.0,NaN,NaN,None,2.0,NaN
6,36,4,1,104,2,1364,10017,1,4,PINAPPLE,...,None,13000.0,2.0,NaN,NaN,13.0,81.0,None,2.0,NaN
7,41,4,1,104,2,1364,10018,1,5,PLANTAIN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
8,45,4,1,104,2,1364,10019,2,3,PLANTAIN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
9,52,4,1,106,2,880,10021,4,1,CASSAVA,...,None,10000.0,2.0,NaN,NaN,4.0,131.0,None,2.0,NaN
